In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

movie=pd.read_csv('dataset/tmdb_5000_movies.csv')
credits=pd.read_csv('dataset/tmdb_5000_credits.csv')

In [2]:
movie.head
movie.shape



(4803, 20)

In [3]:
movie=movie.merge(credits,on='title')
movie.shape


(4809, 23)

In [4]:
movie.columns


Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'movie_id', 'cast', 'crew'],
      dtype='object')

In [5]:
movie=movie[['movie_id','title','overview','genres', 'keywords','cast', 'crew','original_language',]]


In [6]:
movie.isnull().sum()


movie_id             0
title                0
overview             3
genres               0
keywords             0
cast                 0
crew                 0
original_language    0
dtype: int64

In [7]:
movie.dropna(inplace=True)
movie.shape

(4806, 8)

In [8]:
import ast
def convert(text):
    l=[]
    for i in ast.literal_eval (text): #converting string into list
        l.append(i['name'])
    return l

def convert_c(text):
    l=[]
    c=0
    for i in ast.literal_eval (text):#converting string into list
        if c<5:
         l.append(i['name'])
        c+=1
    
    return l

def convert_cr(text):
    l=[]
  
    for i in ast.literal_eval (text):#converting string into list
        if i['job']=='Director':
          l.append(i['name'])
          break
    
    return l



    

In [9]:
movie['genres']=movie['genres'].apply(convert)
movie['keywords']=movie['keywords'].apply(convert)
movie['cast']=movie['cast'].apply(convert_c)
movie['crew']=movie['crew'].apply(convert_cr)

In [10]:
movie.head(2)

,movie_id,title,overview,genres,keywords,cast,crew,original_language
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weave...",[James Cameron],en
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley, ...",[Gore Verbinski],en


In [11]:
movie['overview']=movie['overview'].apply(lambda x:x.split())
movie.head(2)

,movie_id,title,overview,genres,keywords,cast,crew,original_language
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weave...",[James Cameron],en
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley, ...",[Gore Verbinski],en


In [12]:
#remove space from name
def remove_space(word):
    l=[]
    for i in word:
        l.append(i.replace(" ",""))

    return l

In [13]:
movie['genres']=movie['genres'].apply(remove_space)
movie['keywords']=movie['keywords'].apply(remove_space)
movie['cast']=movie['cast'].apply(remove_space)
movie['crew']=movie['crew'].apply(remove_space)


In [14]:
movie['tags'] = movie['overview'] + movie['genres'] + movie['keywords'] + movie['cast'] + movie['crew']

In [15]:
movie.head(2)

,movie_id,title,overview,genres,keywords,cast,crew,original_language,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver, ...",[JamesCameron],en,"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley, Ste...",[GoreVerbinski],en,"[Captain, Barbossa,, long, believed, to, be, d..."


In [16]:
movies = movie[['movie_id','title','tags']]

In [17]:
movies["tags"] = movies["tags"].apply(lambda x: " ".join(map(str, x)))

# Display first few rows
movies.head()

C:\Users\shaht\AppData\Local\Temp\ipykernel_21476\915764605.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies["tags"] = movies["tags"].apply(lambda x: " ".join(map(str, x)))


,movie_id,title,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...
4,49529,John Carter,"John Carter is a war-weary, former military ca..."


In [18]:
movies['tags'] = movies['tags'].apply(lambda x:x.lower())

C:\Users\shaht\AppData\Local\Temp\ipykernel_21476\1170348455.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['tags'] = movies['tags'].apply(lambda x:x.lower())


In [19]:
movies.head(2)

,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believed to be dead, ha..."


In [20]:
import nltk
from nltk.stem import PorterStemmer
ps=PorterStemmer()

In [21]:
def steam(text):
    l=[]
    for i in text.split():
        l.append(ps.stem(i))

    return " ".join(l)
        

In [22]:
movies['tags'] = movies['tags'].apply(steam)

C:\Users\shaht\AppData\Local\Temp\ipykernel_21476\2966009993.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['tags'] = movies['tags'].apply(steam)


In [23]:
movies.iloc[0]['tags']

'in the 22nd century, a parapleg marin is dispatch to the moon pandora on a uniqu mission, but becom torn between follow order and protect an alien civilization. action adventur fantasi sciencefict cultureclash futur spacewar spacecoloni societi spacetravel futurist romanc space alien tribe alienplanet cgi marin soldier battl loveaffair antiwar powerrel mindandsoul 3d samworthington zoesaldana sigourneyweav stephenlang michellerodriguez jamescameron'

In [24]:
#vectorization after that cosine similarity to find nearest vector as suggested
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000,stop_words='english')

In [25]:
vector=cv.fit_transform(movies['tags']).toarray()

In [26]:
vector.shape


(4806, 5000)

In [27]:
from sklearn.metrics.pairwise import cosine_similarity

sim=cosine_similarity(vector)
sim.shape

(4806, 4806)

In [28]:
movies[movies['title'] == 'Inception'].index[0]

np.int64(96)

In [29]:
def recomand(movie):
    index_list = movies[movies['title'] == movie].index  # Use the variable, not a string
    if index_list.empty:  # Check if the movie exists
        print(f"Movie '{movie}' not found in the dataset.")
        return
    
    index = index_list[0]  # Get the first matching index
    distances = sorted(list(enumerate(sim[index])), reverse=True, key=lambda x: x[1])
    
    print("Recommended Movies:")
    for i in distances[1:6]:  # Fix variable name from `distance` to `distances`
        print(movies.iloc[i[0]]['title'])  # Fetch movie title based on index


In [31]:
recomand('Spider-Man')

Recommended Movies:
Spider-Man 3
Spider-Man 2
The Amazing Spider-Man 2
Arachnophobia
The Amazing Spider-Man
